In [20]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks
from scipy.signal import find_peaks
from scipy.signal import filtfilt, butter
import scipy
import statistics
from google.colab import files

In [210]:
##Cargar EEG DE PRUEBA
df_Aeeg = pd.read_excel("D_P_Cerrados_T_5.xlsx")
rango_a=7499
rango_b=(df_Aeeg['EEGAF3'].size)
df_Aeeg = df_Aeeg.drop(df_Aeeg.index[range(rango_a,rango_b)])
df_Aeeg=df_Aeeg.drop(["EEGCounter","OriginalTimestamp","EEG.Counter","EEGInterpolated","Timestamp","EEGRawCq", "EEGBattery","EEGBatteryPercent","MarkerIndex","MarkerType","MarkerValueInt","EEGMarkerHardware","CQAF3","CQF7","CQF3","CQFC5","CQT7","CQP7","CQO1","CQO2","CQP8","CQT8","CQFC6","CQF4","CQF8","CQAF4","CQOverall","EQSampleRateQuality","EQOVERALL","EQAF3","EQF7","EQF3","EQFC5","EQT7","EQP7","EQO1","EQO2","EQP8","EQT8","EQFC6","EQF4","EQF8","EQAF4","PMEngagementIsActive","PMEngagementScaled","PMEngagementRaw","PMEngagementMin","PMEngagementMax","PMExcitementIsActive","PMExcitementScaled","PMExcitementRaw","PMExcitementMin","PMExcitementMax","PMLongTermExcitement","PMStressIsActive","PMStressScaled","PMStressRaw","PMStressMin","PMStressMax","PMRelaxationIsActive","PMRelaxationScaled","PMRelaxationRaw","PMRelaxationMin","PMRelaxationMax","PMInterestIsActive","PMInterestScaled","PMInterestRaw","PMInterestMin","PMInterestMax","PMFocusIsActive","PMFocusScaled","PMFocusRaw","PMFocusMin","PMFocusMax","POWAF3Theta","POWAF3Alpha","POWAF3BetaL","POWAF3BetaH","POWAF3Gamma","POWF7Theta","POWF7Alpha","POWF7BetaL","POWF7BetaH","POWF7Gamma","POWF3Theta","POWF3Alpha","POWF3BetaL","POWF3BetaH","POWF3Gamma","POWFC5Theta","POWFC5Alpha","POWFC5BetaL","POWFC5BetaH","POWFC5Gamma","POWT7Theta","POWT7Alpha","POWT7BetaL","POWT7BetaH","POWT7Gamma","POWP7Theta","POWP7Alpha","POWP7BetaL","POWP7BetaH","POWP7Gamma","POWO1Theta","POWO1Alpha","POWO1BetaL","POWO1BetaH","POWO1Gamma","POWO2Theta","POWO2Alpha","POWO2BetaL","POWO2BetaH","POWO2Gamma","POWP8Theta","POWP8Alpha","POWP8BetaL","POWP8BetaH","POWP8Gamma","POWT8Theta","POWT8Alpha","POWT8BetaL","POWT8BetaH","POWT8Gamma","POWFC6Theta","POWFC6Alpha","POWFC6BetaL","POWFC6BetaH","POWFC6Gamma","POWF4Theta","POWF4Alpha","POWF4BetaL","POWF4BetaH","POWF4Gamma","POWF8Theta","POWF8Alpha","POWF8BetaL","POWF8BetaH","POWF8Gamma","POWAF4Theta","POWAF4Alpha","POWAF4BetaL","POWAF4BetaH","POWAF4Gamma"],axis= 1, errors='ignore')
##Importar presion plantar 
df_est_antA = pd.read_excel("D_P_Antero_Cerrados_T_5.xlsx")
df_est_antA.columns = ['Tiempo(s)', 'Antero(mm)']
df_est_antA

##Importar estabilometria 
df_est_latA = pd.read_excel("D_P_Lateral_Cerrados_T_5.xlsx")
df_est_latA.columns = ['XDTiempo(s)XD', 'Lateral(mm)']

df_estA = pd.concat([df_est_antA, df_est_latA], axis = 1)
df_estA = df_estA.drop(["XDTiempo(s)XD","Tiempo(s)"],axis = 1)
df_fullA = pd.concat([df_Aeeg, df_estA], axis = 1)

sf = 128 #frecuencia de sample

EEGAF3= df_Aeeg['EEGAF3']
EEGF7 = df_Aeeg['EEGF7']
EEGF3 = df_Aeeg['EEGF3']
EEGFC5 = df_Aeeg['EEGFC5']
EEGT7 = df_Aeeg['EEGT7']
EEGP7 = df_Aeeg['EEGP7']
EEGO1 = df_Aeeg['EEGO1']
EEGO2 = df_Aeeg['EEGO2']
EEGP8 = df_Aeeg['EEGP8']
EEGT8 = df_Aeeg['EEGT8']
EEGFC6 = df_Aeeg['EEGFC6']
EEGF4 = df_Aeeg['EEGF4']
EEGF8 = df_Aeeg['EEGF8']
EEGAF4 = df_Aeeg['EEGAF4']

#Filtrar altas potencias en frecuencias bajas
#____________________________CANAL 1_________________________________________
lowcut = 6
highcut = 30

nyq= 0.5*sf
low = lowcut/nyq
high = highcut/nyq
order = 4

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_1= scipy.signal.filtfilt(b, a, EEGAF3, axis = 0)

#__________________________CANAL 2__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_2= scipy.signal.filtfilt(b, a, EEGF7, axis = 0)

#__________________________CANAL 3__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_3= scipy.signal.filtfilt(b, a, EEGF3, axis = 0)
#__________________________CANAL 4__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_4= scipy.signal.filtfilt(b, a, EEGFC5, axis = 0)
#__________________________CANAL 5__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_5= scipy.signal.filtfilt(b, a, EEGT7, axis = 0)
#__________________________CANAL 6__________________________________________

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_6= scipy.signal.filtfilt(b, a, EEGP7, axis = 0)
#__________________________CANAL 7__________________________________________

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_7= scipy.signal.filtfilt(b, a, EEGO1, axis = 0)
#__________________________CANAL 8__________________________________________

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_8= scipy.signal.filtfilt(b, a, EEGO2, axis = 0)
#__________________________CANAL 9__________________________________________

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_9= scipy.signal.filtfilt(b, a, EEGP8, axis = 0)
#__________________________CANAL 10__________________________________________

b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_10= scipy.signal.filtfilt(b, a, EEGT8, axis = 0)
#__________________________CANAL 11__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_11= scipy.signal.filtfilt(b, a, EEGFC6, axis = 0)
#__________________________CANAL 12__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_12= scipy.signal.filtfilt(b, a, EEGF4, axis = 0)
#__________________________CANAL 13__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_13= scipy.signal.filtfilt(b, a, EEGF8, axis = 0)
#__________________________CANAL 14__________________________________________


b,a = scipy.signal.butter(order, [low, high], 'bandpass', analog= False)
canal_14= scipy.signal.filtfilt(b, a, EEGAF4, axis = 0)

resolucion = 2/0.5
win = resolucion * sf
freqs1, psd1 = signal.welch(canal_1, sf, nperseg=win)
freqs2, psd2 = signal.welch(canal_2, sf, nperseg=win)
freqs3, psd3 = signal.welch(canal_3, sf, nperseg=win)
freqs4, psd4 = signal.welch(canal_4, sf, nperseg=win)
freqs5, psd5 = signal.welch(canal_5, sf, nperseg=win)
freqs6, psd6 = signal.welch(canal_6, sf, nperseg=win)
freqs7, psd7 = signal.welch(canal_7, sf, nperseg=win)
freqs8, psd8 = signal.welch(canal_8, sf, nperseg=win)
freqs9, psd9 = signal.welch(canal_9, sf, nperseg=win)
freqs10, psd10 = signal.welch(canal_10, sf, nperseg=win)
freqs11, psd11 = signal.welch(canal_11, sf, nperseg=win)
freqs12, psd12 = signal.welch(canal_12, sf, nperseg=win)
freqs13, psd13 = signal.welch(canal_13, sf, nperseg=win)
freqs14, psd14 = signal.welch(canal_14, sf, nperseg=win)

#exportar caracteristicas
max_psd1, max_psd2, max_psd3, max_psd4, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12, max_psd13,max_psd14 = max(psd1),max(psd2),max(psd3),max(psd4),max(psd5),max(psd6),max(psd7),max(psd8),max(psd9),max(psd10),max(psd11),max(psd12),max(psd13),max(psd14)


if ((max_psd13>30) or (max_psd13<1)):
  val_max_canales=[max_psd1, max_psd2, max_psd3, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12]
  val_med=sum(val_max_canales)/len(val_max_canales)
  max_psd13=max_psd12
  max_psd14=max_psd12
  print('C13')

if ((max_psd4>30) or (max_psd4<1)):
  val_max_canales=[max_psd1, max_psd2, max_psd3, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12, max_psd13,max_psd14]
  val_med=sum(val_max_canales)/len(val_max_canales)
  max_psd4=max_psd3
  print('C4')


if max_psd14>30 or max_psd14<1:
  val_max_canales=[max_psd1, max_psd2, max_psd3, max_psd4, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12, max_psd13]
  val_med=sum(val_max_canales)/len(val_max_canales)
  max_psd14=max_psd13
  print('C14')

if max_psd1>30 or max_psd1<1:
  val_max_canales=[max_psd2, max_psd3, max_psd4, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12, max_psd13, max_psd14]
  val_med=sum(val_max_canales)/len(val_max_canales)
  max_psd1=max_psd2
  print('C1')

if (max_psd4<30 and max_psd4>1) and (max_psd14<30 and max_psd14>1) and (max_psd1<30 and max_psd1>1):
  val_max_canales=[max_psd1, max_psd2, max_psd3, max_psd4, max_psd5, max_psd6, max_psd7, max_psd8, max_psd9, max_psd10, max_psd11, max_psd12, max_psd13, max_psd14]
  val_med=sum(val_max_canales)/len(val_max_canales)
  print(val_med)

##norm canales electroencefalografia

ch_1=val_med/max_psd1
ch_2=val_med/max_psd2
ch_3=val_med/max_psd3
ch_4=val_med/max_psd4
ch_5=val_med/max_psd5
ch_6=val_med/max_psd6
ch_7=val_med/max_psd7
ch_8=val_med/max_psd8
ch_9=val_med/max_psd9
ch_10=val_med/max_psd10
ch_11=val_med/max_psd11
ch_12=val_med/max_psd12
ch_13=val_med/max_psd13
ch_14=val_med/max_psd14

max_antero=df_fullA['Antero(mm)'].max()
min_antero=df_fullA['Antero(mm)'].min()
max_lateral=df_fullA['Lateral(mm)'].max()
min_lateral=df_fullA['Lateral(mm)'].min()

if abs(max_antero)>abs(min_antero):
  if abs(max_antero)>abs(max_lateral):
    if abs(max_antero)>abs(min_lateral):
      max_est=abs(max_antero)
      print('1')
    else:
      max_est=(abs(min_lateral))
      print('4')
  else:
     if abs(max_lateral)>abs(min_lateral):
       max_est=abs(max_lateral)
       print('3')
     else:
       max_est=abs(min_lateral)
       print('4')
else:
  if abs(min_antero)>abs(max_lateral):
    if abs(min_antero)>abs(min_lateral):
       max_est=abs(min_antero)
       print('2')
    else:
      max_est=abs(min_lateral)
      print('4')
  else:
    if abs(max_lateral)>abs(min_lateral):
      max_est=abs(max_lateral)
      print('3')
    else:
      max_est=abs(min_lateral)
      print('4')

desv_std_lateral = df_fullA["Lateral(mm)"].std()
desv_std_antero = df_fullA['Antero(mm)'].std()
med_lateral= df_fullA["Lateral(mm)"].mean()
med_antero= df_fullA['Antero(mm)'].mean()
max_canales= [ch_1,ch_2,ch_3,ch_4,ch_5,ch_6,ch_7,ch_8,ch_9,ch_10,ch_11,ch_12,ch_13,ch_14]
desv_std_eeg= statistics.pstdev(max_canales)


caract_arreglo = [ch_1,ch_2,ch_3,ch_4,ch_5,ch_6,ch_7,ch_8,ch_9,ch_10,ch_11,ch_12,ch_13,ch_14,val_med,desv_std_eeg,max_est,desv_std_lateral,desv_std_antero,med_lateral,med_antero]
caracteristicas_df=pd.DataFrame([caract_arreglo],columns=['N_AF3','N_F7','N_F3','N_FC5','N_T7','N_P7','N_O1','N_O2','N_P8','N_T8','N_FC6','N_F4','N_F8','N_AF4','MED_CAN','STD_EEG','MAX_EST','STD_LATERAL','STD_ANTERO','MED_LATERAL','MED_ANTERO'])

caracteristicas_df


C4
C1
6.466072486601552
1


,N_AF3,N_F7,N_F3,N_FC5,N_T7,N_P7,N_O1,N_O2,N_P8,N_T8,...,N_F4,N_F8,N_AF4,MED_CAN,STD_EEG,MAX_EST,STD_LATERAL,STD_ANTERO,MED_LATERAL,MED_ANTERO
0,2.17713,2.17713,1.882663,1.882663,3.368917,1.829542,4.996081,1.468155,0.25699,0.488056,...,0.946166,1.888007,0.809216,6.466072,1.201547,14.664,3.778999,4.961292,-0.190742,-0.126726


In [211]:
tar=0


caract_arreglo = [ch_1,ch_2,ch_3,ch_4,ch_5,ch_6,ch_7,ch_8,ch_9,ch_10,ch_11,ch_12,ch_13,ch_14,val_med,desv_std_eeg,max_est,desv_std_lateral,desv_std_antero,med_lateral,med_antero,tar]
caracteristicas_df=pd.DataFrame([caract_arreglo],columns=['N_AF3','N_F7','N_F3','N_FC5','N_T7','N_P7','N_O1','N_O2','N_P8','N_T8','N_FC6','N_F4','N_F8','N_AF4','MED_CAN','STD_EEG','MAX_EST','STD_LATERAL','STD_ANTERO','MED_LATERAL','MED_ANTERO','TARGET'])


#export
caracteristicas_df.to_csv('df_1.csv', index=False)
files.download('df_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>